# CONCRETE STRENGTH PREDICTION USING ANN

- Regression is a type of supervised machine learning algorithm used to predict a continuous label. The goal is to produce a model that represents the 'best fit' to some observed data, according to an evaluation criterion.


- In this Analysis, you will be predicting the strength of concrete using the given eight features that describe the components used in the mixture.These features are believed to be related to the final compressive strength which include the amount of cement in Kilograms per cubic meter, Blast.Furnace.Slag, Fly.Ash, water,superplasticizer, coarse aggregate, fine aggregate and aging period measured in days.

## Importing Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import  callbacks
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mae
import seaborn as sns
%matplotlib inline

# for Interactive Shells
from IPython.display import display

#removing warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

## Import Dataset

In [2]:
df = pd.read_csv('concrete_data.csv')
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'concrete_data.csv'

The dataset contains 1030 observations accross 8 input variables and an output variable. The variable name, variable type, the measurement unit and a brief description is provided.

In [ ]:
df.head()

### There are 8 independent variables in training set:

- Cement: Total amount of cement it that particular sample.
- Blast Furnace Slag:  Blast furnace slag (BFS) is a byproduct of the iron and steel manufacturing process. BFS is used to improve the strength and durability of concrete.
- FlyAsh :  It improves the strength and durability of the concrete.
- Water: The optimal water content for cement is usually between 15-20% of the total cement weight. Too little water will result in a dry, crumbly mix, while too much water can lead to a weak, porous concrete.
- Superplasticizer:  Superplasticizers, also known as plasticizers or high range water reducers, are a type of admixture that is used to increase the workability of concrete without increasing the water-cement ratio.
- CoarseAggregatet:  Coarse aggregate is used for a variety of purposes, including providing strength, improving the workability of concrete, and reducing the amount of cement needed in a concrete mix.
- FineAggregate:  Similar as CoarseAggregateComponent but smaller in size.
- Age:  Age in days is a term used to describe the amount of time that has passed since the cement was mixed with water. Concrete gains strength over time and its properties can change significantly depending on the age. The most commonly used ages for testing concrete strength are 7, 14, 21, and 28 days.

### Target varibale:

- Strength:  Strength of the cement.

## Exploratory Data Analysis

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
# Summary statistics
df.describe()

All predictors in the dataset are quantitative datatypes. Moreover, there are no missing values in the data. Lets analyse the probability distribution of the variables through a series of univariate density plots.

In [ ]:
#  Getting known with the kind of distrubutions
def dist_custom(dataset, columns_list, rows, cols, suptitle):
    fig, axs = plt.subplots(rows, cols,figsize=(16,16))
    fig.suptitle(suptitle,y=0.92, size=16)
    axs = axs.flatten()
    for i, data in enumerate(columns_list):
        sns.distplot(dataset[data], ax=axs[i])
        axs[i].set_title(data + ', skewness is '+str(round(dataset[data].skew(axis = 0, skipna = True),2)))

In [ ]:
dist_custom(dataset=df, columns_list=df.columns, rows=3, cols=3, suptitle='Distibution for each variable')

Most variables do not follow normal distribution. It would be interesting to convert the response variable into a category of 'optimal' and 'suboptimal' in order to detect any interesting patterns between the predictors and the target variable. I'll conisder the observations at the 75th percentile or greater to be good.

In [ ]:
# outliers
def boxplots_custom(dataset, columns_list, rows, cols, suptitle):
    fig, axs = plt.subplots(rows, cols, sharey=True, figsize=(16,12))
    fig.suptitle(suptitle,y=0.93, size=16)
    axs = axs.flatten()
    for i, data in enumerate(columns_list):
        if i % 3 == 0:
            axs[i].set_ylabel('The number of entries')
        sns.boxplot( data=dataset[data], orient='h', ax=axs[i])
        axs[i].set_title(data)

In [ ]:

boxplots_custom(dataset=df, columns_list=df.columns, rows=3, cols=3, suptitle='Boxplots before deleting outliers')

In [ ]:
# calculating the correlation matrix
corr = df.corr()
matrix = np.triu(corr)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, vmax=1.0, vmin=-1.0, 
            fmt='.1g', annot=True, mask = matrix)

plt.title('Correlation matrix', size=16)
plt.show()

The pearsons correlation assumes normality, linearity, homoscedasticity and no outliers. These assumptions are violated for most variables. This may be a potential reason as to why pearsons correlation won't detect nuances between predictors and the independent variables properly. Additionally, the relationship between inputs does seem to be largely non-linear. Nonetheless, cement, water, and superplastisizer do seem to be moderately correlated with strength.

In [ ]:
sns.pairplot(data=df,diag_kind="hist")
plt.show()

## Data Preprocessing

Some machine learning algorithms require features to be scaled through normalization or standardization. I will now split the dataset into training and testing and then perform standardization. This will tansform the features such that their mean and standard deviation will become 0 and 1.

In [ ]:

X = df.iloc[:,0:-1].values
y = df.iloc[:,-1].values

In [ ]:
# Split the dataset into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 43)

In [ ]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test =sc.transform(X_test)

## Model Building using ANN

In [ ]:
# Improving the ANN
from tensorflow.keras.layers import Dropout
model1 = Sequential()

# Adding the input layer and the first hidden layer
model1.add(Dense(units = 100, 
                     kernel_initializer = 'uniform', 
                     activation = 'relu', 
                     input_shape = [X_train.shape[1]]))
               

# Adding the second hidden layer
model1.add(Dense(units = 60, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the Third hidden layer
model1.add(Dense(units = 80, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the fourth hidden layer
model1.add(Dense(units = 90, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the fifth hidden layer
model1.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
model1.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [ ]:
model1.summary()

In [ ]:
# Compiling the ANN
model1.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
history = model1.fit(X_train,y_train,
                          batch_size=62 , 
                          epochs=100 , 
                          validation_split=0.1,
                          verbose = 1)

In [ ]:
history = pd.DataFrame(history.history)
history.loc[:, ['loss']].plot();
history.loc[:, ['val_loss']].plot();

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
MAE = mae(y_test,y_pred)

plt.figure(figsize=(12,8))
with plt.style.context('fivethirtyeight'):

    plt.plot(sorted(y_test), label='Actual')
    plt.plot(sorted(y_pred), label='Predicted')
#     plt.fill_between(x=np.arange(0,len(y_pred)),
#                      y1=np.array(sorted(y_pred)+MAE),
#                      y2=sorted(y_pred)-MAE, 
#                      alpha=0.1, color='r', label='MAE')

    plt.title('Testing prediction\nMean Absolute Error = {:.3f}'.format(MAE))
    plt.ylabel('Concrete Strength')
    plt.xlabel('Item')
    plt.legend()
plt.show()

This model is not performing better so we need to fine tune this model for better accuracy 

## Model Tuning 

## Improving with different architecture

In [ ]:
model2 = keras.Sequential([
    Dense(40 , activation = 'relu' , input_shape = [X_train.shape[1]]),
    Dense(50 , activation = 'relu'),
    Dense(60 , activation = 'relu'),
    Dense(1 , activation = 'linear')
])

In [ ]:
# summary of the model
model2.summary()

In [ ]:
model2.compile(optimizer='Adam', loss = 'mean_squared_error' , )

In [ ]:
history = model2.fit(X_train,y_train,batch_size=62 , epochs=100 , validation_split=.1,verbose = 1)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss']].plot();
history_df.loc[:, ['val_loss']].plot();

## Prediction on Unseen Data

In [ ]:
y_pred = model2.predict(X_test)

## Model Evaluation 

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
MAE = mae(y_test,y_pred)

plt.figure(figsize=(12,8))
with plt.style.context('fivethirtyeight'):

    plt.plot(sorted(y_test), label='Actual')
    plt.plot(sorted(y_pred), label='Predicted')
#     plt.fill_between(x=np.arange(0,len(y_pred)),
#                      y1=np.array(sorted(y_pred)+MAE),
#                      y2=sorted(y_pred)-MAE, 
#                      alpha=0.1, color='r', label='MAE')

    plt.title('Testing prediction\nMean Absolute Error = {:.3f}'.format(MAE))
    plt.ylabel('Concrete Strength')
    plt.xlabel('Item')
    plt.legend()
plt.show()

## Improving ANN with Dropout layer

In [ ]:
model3 = keras.Sequential([
    Dense(30 , activation = 'relu' , input_shape = [X_train.shape[1]]),
    Dense(40 , activation = 'relu'),
    Dropout(rate = 0.1),
    Dense(50 , activation = 'relu'),
    Dense(1 , activation = 'linear')
])



In [ ]:
model3.summary()

In [ ]:
# Compiling the ANN
model3.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
history = model3.fit(X_train,y_train,
                          batch_size=62 , 
                          epochs=100 , 
                          validation_split=0.1,
                          verbose = 1)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss']].plot();
history_df.loc[:, ['val_loss']].plot();

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
MAE = mae(y_test,y_pred)

plt.figure(figsize=(12,8))
with plt.style.context('fivethirtyeight'):

    plt.plot(sorted(y_test), label='Actual')
    plt.plot(sorted(y_pred), label='Predicted')
#     plt.fill_between(x=np.arange(0,len(y_pred)),
#                      y1=np.array(sorted(y_pred)+MAE),
#                      y2=sorted(y_pred)-MAE, 
#                      alpha=0.1, color='r', label='MAE')

    plt.title('Testing prediction\nMean Absolute Error = {:.3f}'.format(MAE))
    plt.ylabel('Concrete Strength')
    plt.xlabel('Item')
    plt.legend()
plt.show()

# Conclusion 

We trained three ANN model using Different Fine Tuning Technique, from all the three models 2nd model is giving better performance so will select the second model for Deployment